# Demo: OpenAI Large Language Model - A RAG Sample

In this demo, we show how to use GPT3 model to analyze natural query, use knowledge base to search for more information and answer questions.

In [1]:
import openai
import os
from dotenv import load_dotenv
load_dotenv()

openai.api_type = "azure"
openai.api_version = os.getenv("OPENAI_API_VERSION")
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.api_base = os.getenv("OPENAI_API_BASE")

CHAT_COMPLETIONS_MODEL = os.getenv('GPT_4_DEPLOYMENT')

### Start with a natural question

In [2]:
input = "Which presendential candidate is likely to win in 2024 election? Explain the reasons."

### Step 1: GPT3: What do I need to to answer this question?

In [3]:
system_prompt=f'''Answer the following questions as best you can. You have access to the following tools:

Web Search: Use the web to find information

You need to follow STRICTLY below format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Research]
Action Input: the input to the action

Begin!
'''
user_prompt = f'''
Question: {input}
Thought:'''

In [4]:
response = openai.ChatCompletion.create(
    engine=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":system_prompt},
                {"role":"user","content": user_prompt,}],
        max_tokens=400,)

print(response['choices'][0]['message']['content'])

This question involves predicting future events which are uncertain and may involve opinions. As of now, it would be appropriate to research the popularity of current politicians, potential candidates, policies, or political trends. However, it is important to note that the answer might not be fully accurate due to the unpredictable nature of politics and human decisions.
Action: Research
Action Input: 2024 presidential election predictions


### Step 2: Search web for more details 

In [12]:
# Use Azure BING Search API to get the results
import requests
import json
url = "https://api.bing.microsoft.com/v7.0/search"
headers = {"Ocp-Apim-Subscription-Key": os.getenv('BING_SUBSCRIPTION_KEY')}
params = {"q": "2024 presidential election candidates and current political analysis"}
response = requests.get(url, headers=headers, params=params)

In [13]:
print(response.json())

{'_type': 'SearchResponse', 'queryContext': {'originalQuery': '2024 presidential election candidates and current political analysis'}, 'webPages': {'webSearchUrl': 'https://www.bing.com/search?q=2024+presidential+election+candidates+and+current+political+analysis', 'totalEstimatedMatches': 734000, 'value': [{'id': 'https://api.bing.microsoft.com/api/v7/#WebPages.0', 'name': 'The 2024 presidential candidates: Who’s running from both sides', 'url': 'https://www.nbcnews.com/politics/2024-elections/presidential-candidates-tracker', 'thumbnailUrl': 'https://www.bing.com/th?id=OIP.qhumvfIio6sQHsnltmAI-wAAAA&w=80&h=80&c=1&pid=5.1', 'datePublished': '2024-02-08T15:51:00.0000000', 'datePublishedDisplayText': 'Feb 8, 2024', 'isFamilyFriendly': True, 'displayUrl': 'https://www.nbcnews.com/politics/2024-elections/presidential-candidates-tracker', 'snippet': 'As voters in early states head to the polls, the Republican presidential field has dropped to two candidates. Former President Donald Trump c

In [14]:
context = ''
for result in response.json()['webPages']['value']:
    context += result['snippet'] + '\n'

### Collected information for further processing

In [15]:
print(context)

As voters in early states head to the polls, the Republican presidential field has dropped to two candidates. Former President Donald Trump continues to lead the pack by a considerable margin and ...
Campaigns 2016 to 2024: When Candidates Entered and Exited Their Races. In the past two presidential election cycles, most announced their bids by summer the year before Election Day and dropped ...
2024 election guide: Presidential candidates, polls, primaries and caucuses, voter information and results for November 5, 2024
Election 2024. Get the latest news on the 2024 election. Compare where the 2024 presidential candidates stand on key issues like abortion, climate and the economy.
Live 2024 election results and maps by state The presidential nominating contests begin with the Iowa caucuses on Jan. 15 and the first-in-the-nation New Hampshire primary on Jan. 23.
The latest political polls and polling averages from FiveThirtyEight. UPDATED Feb. 22, 2024, at 3:59 PM ... An updating avera

In [17]:
system_prompt=f'''Answer the following questions as best you can. Use the context provided below:
{context}

Begin!
'''
user_prompt = f'''
Question: {input}\
Answer:'''

### Step 3: GPT3: What do I need to do to answer this question? (Again)

In [18]:
response = openai.ChatCompletion.create(
    engine=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":system_prompt},
                {"role":"user","content": user_prompt,}],
        max_tokens=400,)

print(response['choices'][0]['message']['content'])

The text does not provide specific information on which candidate is likely to win the 2024 presidential election. The race is noted as being between former President Donald Trump, leading the Republican pack, and the incumbent president, Joe Biden, as the presumptive Democratic nominee. The results will depend on various factors including their campaign strategies, current political climate, and voters' perception of their stance on key issues.


### GPT3: I now have final answer to the question. 